# import libraries

In [6]:
import tensorflow as tf
import time
# Check TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

# List available physical devices
physical_devices = tf.config.list_physical_devices('GPU')
print("Available GPUs:")
for device in physical_devices:
    print(device)
    
# Check if TensorFlow is using the GPU
if tf.test.is_built_with_cuda():
    print("TensorFlow is built with CUDA")
    if len(physical_devices) > 0:
        print("TensorFlow is using the GPU")
    else:
        print("TensorFlow is not using the GPU, but it is available")
else:
    print("TensorFlow is not built with CUDA")


TensorFlow version: 2.6.0
Available GPUs:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
TensorFlow is built with CUDA
TensorFlow is using the GPU


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint,Callback,LearningRateScheduler , TensorBoard
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras import backend as K
import os
import datetime
import numpy as np
import shutil
# from skimage import morphology, feature, segmentation, measure, io, color
# from skimage.filters import sobel
import matplotlib.pyplot as plt
import cv2
import pdb
import imageio
from scipy import misc
import scipy.ndimage as ndimage
import warnings
warnings.filterwarnings('ignore')
# from google.colab.patches import cv2_imshow  # Only required in Google Colab


# Generating DIC like images

In [8]:
base_address= 'C:/Users/narges/PycharmProjects3/pythonProject3/IEEE_transaction_paper'
# Setting dimensions
image_height = 256
image_width = 256

In [9]:
folder_path = base_address +'/real_dataset/data_80_cells/'
# output_folder_path = base_address+'/real_dataset/data_80_dics/'
output_file_path = base_address + '/visulize the preprocessing steps/DIC_samples/'
# files = os.listdir(folder_path)
files=['077cell.png']
for file in files:
    # Create the full file path
    print(os.path.join(folder_path, file))
    if 'ipynb_checkpoints' not in os.path.join(folder_path, file) :

        file_path = os.path.join(folder_path, file)
        # Load the input image
        input_image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    
        # Apply gradient filter to find edges of the cells
        gradient_x = cv2.Sobel(input_image, cv2.CV_64F, 1, 0, ksize=3)
        gradient_y = cv2.Sobel(input_image, cv2.CV_64F, 0, 1, ksize=3)
        gradient_magnitude = cv2.magnitude(gradient_x, gradient_y)
    
        # Normalize gradient magnitude to [0, 255]
        gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX)
    
        # Simulate shading
        shading_strength = 0.5 # Adjust this parameter to control shading strength
        shading = np.ones_like(input_image, dtype=np.float64) * 127
        shading += shading_strength * gradient_magnitude
    
        # Add shading to the input image
        output_image = cv2.add(input_image, shading.astype(np.uint8))
    
        
        cv2.imwrite(output_file_path+ file.replace("cell.png" , "dic.png") , output_image)

C:/Users/narges/PycharmProjects3/pythonProject3/IEEE_transaction_paper/real_dataset/data_80_cells/077cell.png


# Generating Centroids labels

In [10]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Set paths
label_image_folder = base_address + '\\real_dataset\\data_80_dots\\'
label_path = os.path.join(label_image_folder, '077dots.png')

print(label_path)
# Step 1: Read label image (Ground truth)
label_image = cv2.imread(label_path)
plt.imshow(cv2.cvtColor(label_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
# plt.title("Step 1: Label Image")
plt.savefig(base_address + '/visulize the preprocessing steps/centroids_samples/step1_label_image.png', dpi=300  ,bbox_inches='tight',transparent=True, pad_inches=0)
plt.close()
# cv2.imwrite( base_address + '/step1_label_image.png' , label_image.astype(np.uint8) )

# Step 2: Process label image for centers (Binary mask)
label_image1 = label_image.copy()  # Keep the original label image for later use
# Create a binary mask where the cell centers are highlighted (non-zero)
label_image =  label_image[:, :, 2] > 0  # Apply threshold to the red channel

plt.imshow(label_image, cmap='gray')
plt.axis('off')
# plt.title("Step 3: Highlighted Centers (Binary Mask)")
plt.savefig(base_address + '/visulize the preprocessing steps/centroids_samples/step2_highlighted_centers.png', dpi=300 ,bbox_inches='tight',transparent=True, pad_inches=0)
plt.close()
# cv2.imwrite( base_address + '/step2_highlighted_centers.png'  , label_image.astype(np.uint8) )

factor =100
# Step 3: Magnified centers (Apply magnification factor)
magnified_label_image = factor * (label_image1[:, :, 2] > 0)  # Magnified centers
plt.imshow(magnified_label_image, cmap='gray')
plt.axis('off')
# plt.title("Step 4: Magnified Centers")
plt.savefig(base_address + '/visulize the preprocessing steps/centroids_samples/step3_magnified_centers.png', dpi=300 ,bbox_inches='tight',transparent=True, pad_inches=0)
plt.close()
# cv2.imwrite( base_address + '\step3_magnified_centers.png' , label_image.astype(np.uint8))


# Step 5: Final label image (Expanded dimensions for processing)
label_image = np.expand_dims(label_image, axis=-1)
plt.imshow(label_image.squeeze(), cmap='gray')
plt.axis('off')
# plt.title("Step 4: Final Label Image (Expanded Dimensions)")
plt.savefig(base_address + '/visulize the preprocessing steps/centroids_samples/step4_final_label_image.png', dpi=300 ,bbox_inches='tight',transparent=True, pad_inches=0)
plt.close()
# cv2.imwrite(base_address + '\step4_final_label_image.png' , label_image.astype(np.uint8))

# Optional: Count the dots
dots_count = str(np.sum(label_image1[:, :, 2] > 0))
print("Dot Count:", dots_count)


C:/Users/narges/PycharmProjects3/pythonProject3/IEEE_transaction_paper\real_dataset\data_80_dots\077dots.png
Dot Count: 432


# Generating regions images

In [11]:

# Set paths
# Step 1 reading image
label_image_folder = base_address + '\\real_dataset\\data_80_dots\\'
label_path = os.path.join(label_image_folder, '077dots.png')


# read the original image
binary_image = cv2.imread( label_path , cv2.IMREAD_GRAYSCALE)
print(binary_image.shape)
black_image = np.zeros((image_height, image_width, 1), dtype=np.uint8)

# Step 2 identify the centroids locations
white_pixel_locations = np.where(binary_image >0)
# Step 3 draw circles with 7 pixel diameter at the centroids locations
for y, x in zip(*white_pixel_locations):
  cv2.circle(black_image, (x, y), 7 , 255, -1)  # Draw a red circle at each white pixel location
  cv2.circle(binary_image, (x, y), 7 , 255, -1)  # Draw a red circle at each white pixel location

# Step 4 save the image 
cv2.imwrite( base_address + '/visulize the preprocessing steps/regions_samples/077_regions.png'  , black_image )



(256, 256)


True

In [12]:
print('  x , y  ')
for i in range(len(white_pixel_locations[0])):
    print("[" , white_pixel_locations[0][i] ,",", white_pixel_locations[1][i] , "]" )

  x , y  
[ 1 , 73 ]
[ 1 , 237 ]
[ 2 , 105 ]
[ 3 , 222 ]
[ 4 , 51 ]
[ 5 , 79 ]
[ 5 , 85 ]
[ 5 , 99 ]
[ 5 , 168 ]
[ 7 , 2 ]
[ 7 , 179 ]
[ 7 , 202 ]
[ 8 , 56 ]
[ 8 , 109 ]
[ 8 , 132 ]
[ 10 , 16 ]
[ 11 , 9 ]
[ 11 , 116 ]
[ 11 , 239 ]
[ 12 , 138 ]
[ 12 , 249 ]
[ 13 , 64 ]
[ 13 , 171 ]
[ 14 , 156 ]
[ 14 , 212 ]
[ 16 , 150 ]
[ 18 , 23 ]
[ 18 , 89 ]
[ 19 , 109 ]
[ 19 , 141 ]
[ 20 , 5 ]
[ 20 , 65 ]
[ 21 , 163 ]
[ 21 , 252 ]
[ 24 , 153 ]
[ 24 , 170 ]
[ 25 , 193 ]
[ 26 , 29 ]
[ 27 , 200 ]
[ 28 , 9 ]
[ 28 , 113 ]
[ 29 , 50 ]
[ 29 , 104 ]
[ 29 , 121 ]
[ 29 , 244 ]
[ 30 , 182 ]
[ 31 , 151 ]
[ 31 , 159 ]
[ 34 , 28 ]
[ 35 , 54 ]
[ 35 , 138 ]
[ 35 , 251 ]
[ 36 , 34 ]
[ 36 , 211 ]
[ 37 , 121 ]
[ 38 , 16 ]
[ 38 , 70 ]
[ 38 , 99 ]
[ 39 , 64 ]
[ 41 , 28 ]
[ 41 , 57 ]
[ 41 , 226 ]
[ 42 , 124 ]
[ 42 , 133 ]
[ 42 , 171 ]
[ 43 , 3 ]
[ 44 , 139 ]
[ 46 , 82 ]
[ 46 , 157 ]
[ 46 , 206 ]
[ 46 , 216 ]
[ 46 , 232 ]
[ 47 , 251 ]
[ 50 , 73 ]
[ 50 , 240 ]
[ 52 , 87 ]
[ 52 , 195 ]
[ 53 , 7 ]
[ 53 , 68 ]
[ 53 , 129 ]
[ 5